In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

'''import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))'''

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

"import os\nfor dirname, _, filenames in os.walk('/kaggle/input'):\n    for filename in filenames:\n        print(os.path.join(dirname, filename))"

In [2]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
from sklearn.model_selection import train_test_split

In [3]:
class data (Dataset):
    def __init__(self,csv,root_dir='', transform=None):
        self.data = pd.read_csv(csv)
        self.root_dir = root_dir
        self.transform = transform
    def __len__(self):
        return len(self.data)
    def __getitem__(self, index):
        img_path = os.path.join(self.root_dir, self.data.iloc[index]['file_name'])
        image = Image.open(img_path).convert('RGB')
        label = int(self.data.iloc[index]['label'])
        if self.transform:
            image = self.transform(image)
        return image, label

In [4]:
train_transforms = transforms.Compose([
    transforms.Resize((518,518)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],  
                         [0.229, 0.224, 0.225])
    
])
val_transforms = transforms.Compose([
    transforms.Resize((518,518)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],  
                         [0.229, 0.224, 0.225])
    
])

In [5]:
csv_path = '/kaggle/input/detect-ai-vs-human-generated-images/train.csv'

df = pd.read_csv(csv_path)
train_df, val_df = train_test_split(df, test_size=0.4, random_state=42, stratify=df['label'])

train_df.to_csv('train_split.csv', index=False)
val_df.to_csv('val_split.csv', index=False)

In [6]:
train_dataset = data(
    csv = 'train_split.csv',
    root_dir = '/kaggle/input/ai-vs-human-generated-dataset',
    transform = train_transforms
)
val_dataset = data(
    csv = 'val_split.csv',
    root_dir = '/kaggle/input/ai-vs-human-generated-dataset',
    transform = val_transforms
    
)

In [7]:
print(f"Number of training samples: {len(train_dataset)}")
print(f"Number of validation samples: {len(val_dataset)}")


Number of training samples: 47970
Number of validation samples: 31980


In [8]:
batch_size = 32
train_loader = DataLoader(
    train_dataset,
    batch_size = batch_size,
    shuffle = True,
    num_workers = 4
)
val_loader = DataLoader(
    val_dataset,
    batch_size = batch_size,
    shuffle = False,
    num_workers = 4
)

In [9]:
temp = pd.read_csv('train_split.csv')
temp

,Unnamed: 0,file_name,label
0,69979,train_data/bc5e653109f14d778f578e09ec29dbcd.jpg,0
1,12831,train_data/f913d2c7cb7b4c58a328c1e2024a2750.jpg,0
2,74697,train_data/7f448ab672934fb89648f878e1fca8a1.jpg,0
3,50571,train_data/ab787e86080443819180e8d6ab10e6da.jpg,0
4,26683,train_data/b6dfda3fb59d41a0a6d3d28d2f39c577.jpg,0
...,...,...,...
47965,20179,train_data/f400ab0d6b7e4d3f85101099fda03849.jpg,0
47966,47744,train_data/e1eee53fa4294c2a832edad57ccc177e.jpg,1
47967,2758,train_data/ff393399af4e401eabcad9ac6a6ed7f6.jpg,1
47968,62613,train_data/9dae7abd88cc4bd0b42af88c44b598e3.jpg,0


In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from timm import create_model
from torch.optim.lr_scheduler import StepLR

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


model = create_model('vit_small_patch14_dinov2.lvd142m', pretrained=True, num_classes=2)


if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs")
    model = nn.DataParallel(model)


model = model.to(device)


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-5, weight_decay=1e-2)
scheduler = StepLR(optimizer, step_size=1, gamma=0.1)

print(model)


model.safetensors:   0%|          | 0.00/88.2M [00:00<?, ?B/s]

Using 2 GPUs
DataParallel(
  (module): VisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 384, kernel_size=(14, 14), stride=(14, 14))
      (norm): Identity()
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (patch_drop): Identity()
    (norm_pre): Identity()
    (blocks): Sequential(
      (0): Block(
        (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=384, out_features=1152, bias=True)
          (q_norm): Identity()
          (k_norm): Identity()
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=384, out_features=384, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (ls1): LayerScale()
        (drop_path1): Identity()
        (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=384, out_features=1536, bias=True)
          (act): GELU(app

In [11]:
def train_one_epoch(model, criterion, optimizer, dataloader, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in dataloader:
        inputs,labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs,1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_accuracy = correct / total
    return epoch_loss, epoch_accuracy


def validate_one_epoch(model, criterion, optimizer, dataloader, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad(): 
         for inputs, labels in dataloader:
                   inputs,labels = inputs.to(device), labels.to(device)
                   outputs = model(inputs)
                   loss = criterion(outputs, labels)
                   running_loss += loss.item() * inputs.size(0)
                   _, preds = torch.max(outputs,1)
                   correct += (preds == labels).sum().item()
                   total += labels.size(0)
        

    epoch_loss = running_loss / total
    epoch_accuracy = correct / total
    return epoch_loss, epoch_accuracy
    

In [12]:
num_epochs = 1
best_val_accuracy = 0.0
save_path = "best_model.pth"

for epoch in range(num_epochs):
    train_loss, train_accuracy = train_one_epoch(model, criterion, optimizer, train_loader, device)
    val_loss, val_accuracy = validate_one_epoch(model, criterion, optimizer, val_loader, device)
    print(f"Epoch [{epoch+1}]"
          f" Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}"
          f" Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")
    
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        print(f"Saving best model with Validation Accuracy: {best_val_accuracy:.4f}")
        torch.save(model.state_dict(), save_path)

print(f"Best Validation Accuracy: {best_val_accuracy:.4f}")


Epoch [1] Train Loss: 0.0410, Train Accuracy: 0.9842 Validation Loss: 0.0090, Validation Accuracy: 0.9969
Saving best model with Validation Accuracy: 0.9969
Best Validation Accuracy: 0.9969


In [13]:
class data (Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform
    def __len__(self):
        return len(self.df)
    def __getitem__(self, index):
        img_path = '/kaggle/input/ai-vs-human-generated-dataset/' + self.df.iloc[index]['id']
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image

In [14]:
# /kaggle/input/ai-vs-human-generated-dataset/test_data_v2/0016e1d72d404fe68074cc87cb30aa37.jpg

In [15]:
test = pd.read_csv('/kaggle/input/detect-ai-vs-human-generated-images/test.csv')
test_dataset = data(
    test,
    transform = val_transforms
)
test_loader = DataLoader(test_dataset, batch_size = 32, shuffle = False)

In [16]:
print(f"Number of training samples: {len(test_dataset)}")

Number of training samples: 5540


In [17]:
best_model_path = "best_model.pth"
model.load_state_dict(torch.load(best_model_path))
model.to(device)
model.eval()

preds = []
with torch.no_grad():
    for images in test_loader:
        images = images.to(device)
        outputs = model(images)
        _, pred = torch.max(outputs, 1)
        reversed_pred = 1 - pred  
        preds.extend(reversed_pred.cpu().numpy().tolist())

submission_df = pd.DataFrame({
    'id': test['id'],
    'label': preds
})

submission_df.to_csv('submission.csv', index=False)

print("Reversed predictions saved to submission.csv")


<ipython-input-17-16872eb5bbba>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(best_model_path))


Reversed predictions saved to submission.csv
